In [1]:
# show result of last expression even if it was an assignment
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

# Step 1: The Station List

**Option 1:** Use the `gbfs-client` package to fetch the Greenbike GBFS feed: https://pypi.org/project/gbfs-client/

In [2]:
GBFS_URL = "https://gbfs.bcycle.com/bcycle_greenbikeslc/gbfs.json"

'https://gbfs.bcycle.com/bcycle_greenbikeslc/gbfs.json'

In [3]:
from gbfs.client import GBFSClient
client = GBFSClient(GBFS_URL)
gbfs_stations = client.request_feed('station_information').get('data').get('stations')

[{'lon': -111.88774,
  'lat': 40.76704,
  'rental_uris': {'ios': 'https://www.bcycle.com/applink?system_id=bcycle_greenbikeslc&station_id=bcycle_greenbikeslc_2248&platform=iOS',
   'android': 'https://www.bcycle.com/applink?system_id=bcycle_greenbikeslc&station_id=bcycle_greenbikeslc_2248&platform=android'},
  '_bcycle_station_type': 'Kiosk and Station',
  'address': '123 E 100 S',
  'name': 'Harmons Station @ 123E 100S',
  'station_id': 'bcycle_greenbikeslc_2248'},
 {'lon': -111.89126,
  'lat': 40.76903,
  'rental_uris': {'ios': 'https://www.bcycle.com/applink?system_id=bcycle_greenbikeslc&station_id=bcycle_greenbikeslc_2251&platform=iOS',
   'android': 'https://www.bcycle.com/applink?system_id=bcycle_greenbikeslc&station_id=bcycle_greenbikeslc_2251&platform=android'},
  '_bcycle_station_type': 'Kiosk and Station',
  'address': '6 S. Main St.',
  'name': 'Key Bank Station @ 6S Main Street',
  'station_id': 'bcycle_greenbikeslc_2251'},
 {'lon': -111.89544,
  'lat': 40.76266,
  'rental_

In [ ]:
# Just in case we can't get the live GBFS data during the presention, this loads
# it from a local snapshot instead.
from gbfs.client import GBFSClient
from gbfs.data.fetchers import LocalJSONFetcher
client = GBFSClient(url="gbfs_snapshot/gbfs.json", json_fetcher=LocalJSONFetcher())
gbfs_stations = client.request_feed('station_information').get('data').get('stations')

**Option 2:** Use the `pybikes` Python package: https://github.com/eskerda/PyBikes

In [ ]:
import pybikes
system_info = pybikes.get('bicing')
system_info.update()

for station in system_info.stations:
    print(f"{station.name:80}{station.free:>2}")

**After either option:** Store station info in convenient data structure

In [6]:
from typing import NamedTuple

class Station(NamedTuple):
    lat: float
    lon: float
    name: str
    id: str

    @property
    def lonlat(self):
        return self.lon, self.lat
       
    @property
    def latlon(self):
        return self.lat, self.lon
        
    def __repr__(self):
        return (
            f"{self.lat:9.5f} {self.lon:9.5f} {self.id:5} {self.name[:40]}")

In [7]:
stations = [
    Station(
        lat=station["lat"],
        lon=station["lon"],
        name=station["name"],
        id=int(station["station_id"].split("_")[-1]), 
        # e.g. "bcycle_greenbikeslc_1234"
    ) 
    for station in gbfs_stations
    if station.get("region_id") == None  # None = SLC, 13 = Ogden
]

[ 40.76704 -111.88774  2248 Harmons Station @ 123E 100S,
  40.76903 -111.89126  2251 Key Bank Station @ 6S Main Street,
  40.76266 -111.89544  2252 Squatters Station @ 147W Broadway,
  40.76812 -111.89419  2254 20S West Temple Station,
  40.76142 -111.89091  2255 City Weekly Station @ 353S Main Street,
  40.76975 -111.90274  2256 400W South Temple Station,
  40.76430 -111.89095  2257 Rocky Mountain Power Station @ 225S Main,
  40.77181 -111.89667  2258 The Church of Jesus Christ of Latter-Day,
  40.76257 -111.90850  2259 UTA Salt Lake Central Station @ 290S 600,
  40.76906 -111.89707  2400 Radisson Station @ 215W South Temple,
  40.75943 -111.88521  2735 UCAIR Station @ 450S 200E,
  40.77132 -111.90874  2736 640W  North Temple Station,
  40.76624 -111.89124  2737 Energy Development Station @ 136S Main S,
  40.75861 -111.89593  2738 Sheraton Station @ 500S 150W,
  40.76539 -111.87982  2739 CityScape Station @ 190S 400E,
  40.76297 -111.88286  2740 3 & 3 Uncommons Station ,
  40.76283 -1

In [8]:
# As of Oct 2022, there are 42 stations in SLC (and 6 more in Ogden that 
# get filtered out above)

len(stations)  

42

# Step 2: Folium for Mapping

Folium on PyPI: https://pypi.org/project/folium/

In [9]:
import folium

def basemap(stations: list[Station]) -> folium.Map:
    center_lat = 0.5 * (min(st.lat for st in stations) + max(st.lat for st in stations))
    center_lon = 0.5 * (min(st.lon for st in stations) + max(st.lon for st in stations))
    
    mymap = folium.Map(
        location=[center_lat, center_lon], 
        zoom_start=13,
        tiles='CartoDB positron',  # other options: Stamen Terrain, Stamen Toner (high contrast)
    )

    for idx, st in enumerate(stations):
        folium.Circle(
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=1.0,
            location=st.latlon, 
            radius=30,
            popup=f"<strong>#{idx}&nbsp;({st.id})</strong> {st.name}",
        ).add_to(mymap)
        
    return mymap

In [10]:
basemap(stations)

The distance calculator is approximately 1% of the feature set of geopy, but here we go: https://pypi.org/project/geopy/

In [11]:
from geopy.distance import distance
from functools import cache

@cache
def as_the_bird_flies(from_station: Station, to_station: Station):
    """Returns distance between two stations in whole meters"""
    dist = distance(from_station.latlon, to_station.latlon)
    return int(dist.meters)

In [12]:
distance_26_to_29 = as_the_bird_flies(stations[26], stations[29]) / 1609

3.238036047234307

In [17]:
mymap = basemap(stations)
line = folium.PolyLine([stations[26].latlon, stations[29].latlon, stations[15].latlon])
line.add_to(mymap)
mymap

# Step 3: Traveling Salesman with OR-tools

Closely following the "Traveling Salesperson Problem" tutorial: https://developers.google.com/optimization/routing/tsp

In [18]:
from ortools.constraint_solver.pywrapcp import (
    RoutingIndexManager,
    RoutingModel,
    DefaultRoutingSearchParameters,
)

manager = RoutingIndexManager(
    len(stations), # number of stations
    1,             # number of vehicles
    30,            # index of start/finish station
)

<ortools.constraint_solver.pywrapcp.RoutingIndexManager; proxy of <Swig Object of type 'operations_research::RoutingIndexManager *' at 0x109b5f090> >

In [19]:
def cost_function(from_index, to_index):
    """Cost function for OR Tools"""
    from_station: Station = stations[manager.IndexToNode(from_index)]
    to_station: Station = stations[manager.IndexToNode(to_index)]
    return as_the_bird_flies(from_station, to_station)

In [20]:
routing = RoutingModel(manager)
callback_ptr = routing.RegisterTransitCallback(cost_function)
routing.SetArcCostEvaluatorOfAllVehicles(callback_ptr)
search_parameters = DefaultRoutingSearchParameters()
search_parameters.time_limit.seconds = 120

# This line runs the optimization algorithm
solution = routing.SolveWithParameters(search_parameters)

total_dist = solution.ObjectiveValue()
print(f"Distance: {total_dist/1000:.3} km ({total_dist/1609:.3} mi)")

Distance: 22.2 km (13.8 mi)


In [21]:
import itertools

def index_pair_to_polyline(idx1: int, idx2: int) -> folium.PolyLine:
    """Turn two indices from ORtools into a straight line connecting them"""
    st1 = stations[manager.IndexToNode(idx1)]
    st2 = stations[manager.IndexToNode(idx2)]
    return folium.PolyLine([st1.latlon, st2.latlon])

def route_from_ortools_solution(solution, routing) -> list[folium.PolyLine]:
    """Turn OR-tools solution into list of Folium polylines"""
    indices = [routing.Start(0)]
    while not routing.IsEnd(indices[-1]):
        indices.append(solution.Value(routing.NextVar(indices[-1])))
        
    return [
        index_pair_to_polyline(idx1, idx2) 
        for idx1, idx2 in itertools.pairwise(indices)
    ]
    
def draw_route(list_of_stations, list_of_polylines):
    routemap = basemap(list_of_stations)
    for polyline in list_of_polylines:
        polyline.add_to(routemap)
    return routemap
    
route = route_from_ortools_solution(solution, routing)
draw_route(stations, route)

# Step 4: Real cycling directions from Mapbox

In [22]:
import mapbox
from pathlib import Path
MAPBOX_TOKEN = Path("./mapboxtoken.txt").open().read().strip()
service = mapbox.Directions(access_token=MAPBOX_TOKEN)

In [23]:
class RouteSegment(NamedTuple):
    distance: float
    duration: float
    polyline: folium.PolyLine

def get_route_from_mapbox(
    from_station: Station, to_station: Station
) -> RouteSegment:
    """Fetch directions from Mapbox API and turn into Folium polyline"""
    resp = service.directions(
        [from_station.lonlat, to_station.lonlat],
        profile="mapbox/cycling",
        geometries="polyline",
    ).geojson()
    
    feature = resp["features"][0]
    coordinates = feature["geometry"]["coordinates"]
    flipped = [(lat, lon) for (lon, lat) in coordinates]
    
    return RouteSegment(
        distance=feature["properties"]["distance"],
        duration=feature["properties"]["duration"],
        polyline=folium.PolyLine(flipped),
    )

In [24]:
example_route_leg = get_route_from_mapbox(stations[14], stations[15])

RouteSegment(distance=604.1, duration=193.6, polyline=<folium.vector_layers.PolyLine object at 0x10fbabbe0>)

In [25]:
route_segments: dict[tuple[int, int], RouteSegment] = {}

{}

In [ ]:
import itertools
import time

# For the East Bay version I had a cutoff here: If the straight
# line distance is larger than x miles, don't even bother to get
# the cycling distance. This was to keep the request count to 
# Mapbox low and because I didn't want ORTools to even consider
# route legs that long. However, in SLC that doesn't work: 40
# stations are within a one mile radius anyway, but the other two
# stations are more than a mile and a half away from the others.
# Plus, the requests count will only be 42*41 = 1722. Keeping
# the code here but bumping the cutoff to a large number so that  
# it never applies.
CUTOFF_DISTANCE = 999 * 1609

# permutations() and not combinations() because the directions 
# might be different due to one-way streets, etc.
for from_station, to_station in itertools.permutations(stations, 2):
    
    # use straight line distance to filter out unrealistic station
    # pairs
    straight_line = as_the_bird_flies(from_station, to_station)
    if straight_line > CUTOFF_DISTANCE:
        continue

    # call API to get actual cycling route
    route_segments[(from_station.id, to_station.id)] = (
        get_route_from_mapbox(from_station, to_station)
    )

    # rate limit is 300/min = 5/sec
    time.sleep(0.2)  

# Step 5: Traveling Salesman, take 2

In [28]:
def cost_function_2(from_index, to_index):
    """Cost function for OR Tools"""
    st1 = stations[manager.IndexToNode(from_index)]
    st2 = stations[manager.IndexToNode(to_index)]
    
    try:
        segment = route_segments[(st1.id, st2.id)]
        return int(segment.duration)  # in seconds
    
    except KeyError:
        # overestimate duration for routes we don't
        # have directions for (we really want to avoid them)
        return as_the_bird_flies(st1, st2) * 5

In [29]:
routing_2 = RoutingModel(manager)
callback_ptr_2 = routing_2.RegisterTransitCallback(cost_function_2)
routing_2.SetArcCostEvaluatorOfAllVehicles(callback_ptr_2)
search_parameters_2 = DefaultRoutingSearchParameters()
solution_2 = routing_2.SolveWithParameters(search_parameters_2)

total_dist = solution_2.ObjectiveValue()
print(f"Duration: {total_dist/3600:.3} hrs")

Duration: 3.31 hrs


In [30]:
def index_pair_to_polyline(idx1: int, idx2: int) -> folium.PolyLine:
    """Turn two indices from ORtools into a straight line connecting them"""
    st1 = stations[manager.IndexToNode(idx1)]
    st2 = stations[manager.IndexToNode(idx2)]
    # return folium.PolyLine([st1.latlon, st2.latlon])
    return route_segments[st1.id, st2.id].polyline

route_2 = route_from_ortools_solution(solution_2, routing_2)
draw_route(stations, route_2)

Modifying the cost function to penalize rides with durations under 3 minutes a little bit. This cost function considers 3 minutes (180 seconds) ride length the optimum. For example, a two minute ride now has the same cost as a 4 minute ride.

In [31]:
def cost_function_3(from_index, to_index):
    """Cost function for OR Tools - version 3"""
    st1 = stations[manager.IndexToNode(from_index)]
    st2 = stations[manager.IndexToNode(to_index)]
    
    try:
        segment = route_segments[(st1.id, st2.id)]
        return int(abs(180 - segment.duration))  # in seconds
    
    except KeyError as ex:
        return as_the_bird_flies(st1, st2) * 5

In [32]:
routing_3 = RoutingModel(manager)
callback_ptr_3 = routing_3.RegisterTransitCallback(cost_function_3)
routing_3.SetArcCostEvaluatorOfAllVehicles(callback_ptr_3)
search_parameters_3 = DefaultRoutingSearchParameters()
solution_3 = routing_3.SolveWithParameters(search_parameters_3)

<ortools.constraint_solver.pywrapcp.Assignment; proxy of <Swig Object of type 'operations_research::Assignment *' at 0x10fba8c90> >

In [33]:
route_3 = route_from_ortools_solution(solution_3, routing_3)
map_3 = draw_route(stations, route_3)

In [ ]:
map_3.save("extreme-bikeshare-slc-demo.html")